In [18]:
import torch
import torch.nn as nn
import numpy as np
import scipy.io as sio
import time
import os
import argparse
import h5py
from torch.utils.data import TensorDataset, DataLoader
import timeit
from lib.Model_Quant import nnModel


In [19]:
dir_data = 'data'
dir_model = 'model'
ds = 40
lr = 1e-4
gpu = 0
dir_result = 'result'

if not os.path.exists(dir_result):
    os.mkdir(dir_result)

In [20]:
# GPU
GPU_NUM = gpu # GPU number
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device) # change allocation of current GPU
print ('Current cuda device ', torch.cuda.current_device()) # check
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.torch.cuda.memory_reserved(0)/1024**3,1), 'GB')


Current cuda device  0
NVIDIA GeForce MX550
Memory Usage:
Allocated: 0.1 GB
Cached:    0.9 GB


In [21]:
## Dataset #########################################################################
X_mat = sio.loadmat(dir_data + "/input_invivo_test_mtcmrf_PR40.mat")
test_X = X_mat['input_invivo_test']

X_test=torch.FloatTensor(test_X)

print(np.shape(test_X))

#####################################################################################
testset = TensorDataset(X_test)
testloader=DataLoader(testset,batch_size=1,shuffle=False)

## Model loading - Trained model #####
cnn = nnModel(ds,device)
PATH=dir_model+'/NN_model_UL.pth'
checkpoint=torch.load(PATH,map_location=device)
cnn.load_state_dict(checkpoint)
cnn = cnn.to(device)

TEST_DATASIZE = X_test.shape[0]
quantification_result=torch.zeros([TEST_DATASIZE,4,256,256],device=device)

(27, 40, 256, 256)


In [22]:
## test
start_time = timeit.default_timer()


with torch.no_grad(): # important!
    test_loss = 0.0
    for j, data in enumerate(testloader):
        [X_batch]=data
        X_batch = X_batch.to(device)
                        
        x_pred_test = cnn(X_batch)
        quantification_result[j,:,:,:]=x_pred_test

    quantification_result=quantification_result.cpu()
    quantification_result=quantification_result.numpy()

    elapsed = timeit.default_timer() - start_time
    print("Unsupervised pipeline inference took: " + str(elapsed) + " s")
    sio.savemat(dir_result+'/quantification_UL.mat',{'result_nn': quantification_result})


Unsupervised pipeline inference took: 10.290169042011257 s
